## Описание проекта

Перед нами стоит задача защиты данных клиентов страховой компании "Non-prudential plc". Необходимо придумать такой метод преобразования данных, чтобы восстановление исходной информации по трансформированным данным было сложным. Необходимо обосновать корректность его работы.

Защиту нужно провести так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Содержание проекта**

- [1. Загрузка данных](#1.-Загрузка-данных)
- [2. Умножение матриц](#2.-Умножение-матриц)
- [3. Алгоритм преобразования](#3.-Алгоритм-преобразования)
 - [3.1 Описание алгоритма](#3.1-Описание-алгоритма)
 - [3.2 Обоснование алгоритма](#3.2-Обоснование-алгоритма)
- [4. Проверка алгоритма](#4.-Проверка-алгоритма)

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

Загрузим данные и посмотрим на первые строки.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Проверим данные на дубликаты.

In [4]:
len(data[data.duplicated()])

153

Удалим их.

In [5]:
data = data.drop_duplicates().reset_index(drop=True)

Приведем признаки 'Возраст', 'Зарплата' к целочисленному типу.

In [6]:
data[['Возраст','Зарплата']] = data[['Возраст','Зарплата']].astype(int)

Поскольку в ближайшем будущем нам придется иметь дело с матрицами и обучением моделей, создадим новые переменные: X - матрица признаков, y - вектор целей.

In [7]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

In [8]:
X = features.values
X.shape

(4847, 4)

In [9]:
y = target.values
y.shape

(4847,)

## 2. Умножение матриц

В этом задании будем записывать формулы в *Jupyter Notebook.*

Для примера мы записали формулы линейной регрессии.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Здесь нам нужно ответить на вопрос о том, изменится ли качество линейной регрессии, если матрицу признаков умножить на обратимую матрицу.

По логике, нам нужно, чтобы размер получившейся матрицы соответствовал размеру исходной матрицы признаков. Значит, обратимая матрица должна быть квадратная - в нашем случае иметь размер 4 * 4.

Итак, попробуем на формулах посмотреть, что станет с качеством линейной регрессии.

$$
X_1 = X * P
$$

Представим в формуле обучения $X_1$ в виде произведения $X*P$.

$$
w_1 = (X^T_1X_1)^{-1} X^T_1 y = ((XP)^T XP)^{-1} (XP)^T y
$$

Используя свойства транспонированных матриц представим транспонированное произведение $(XP)^T$ как произведение транспонированных матриц, взятых в обратном порядке.

$$
w_1 = (P^TX^TXP)^{-1} P^TX^T y
$$

В первых скобках у нас получилось произведение четырех матриц. Используя свойство ассоциативности, представим это произведение в виде двух множителей - первого и произведения второго, третьего и четвертого.

$$
w_1 = ((P^T)(X^TXP))^{-1} P^TX^T y
$$

Далее мы видим, что получившиеся множители в больших скобках - это квадратные матрицы. 

$P^T$ - квадратная, так как мы определили это в начале, это вытекает из условия нашей задачи.

$X^TXP$ - квадратная, так как произведение матрицы и транспонированной матрицы - квадратная матрица. Потом эта квадратная матрица умножается на квадратную матрицу $P$, в итоге получачется снова квадратная матрица.

Значит, мы можем применить свойство обратных матриц $(AB)^{-1} = B^{-1}A^{-1}$ для квадратных обратимых матриц.

Матрица $P$ - обратимая по условию, значит, мы делаем допущение, что матрица $X^TX$ - обратимая, но в каком-то частном случае она может оказаться необратимой.

Применяем эти свойства и раскрываем скобки.

$$
w_1 = (X^TXP)^{-1} (P^T)^{-1} P^T X^T y
$$

Представим произведение в первых скобках как произведение двух матриц по свойству ассоциативности.

$$
w_1 = ((X^TX)P)^{-1} (P^T)^{-1} P^T X^T y
$$

Снова применяем свойства обратимых квадратных матриц для первых скобок.

$$
w_1 = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y
$$

Поскольку $(P^T)^{-1} P^T = E$, а при умножении любой матрицы на единичную получится эта же матрица, отбросим эти множители.

$$
w_1 = P^{-1} (X^TX)^{-1} X^T y
$$

После этих преобразований мы видим, что наше выражение содержит формулу обучения, значит:

$$
w_1 = P^{-1} w
$$

Проверим по формуле предсказаний $a = Xw$, что с ними произойдет.

$$
a_1 = X_1 w_1 = X P P^{-1} w
$$

Поскольку произведение обратимых матриц равно единичной матрицу, снова отбрасываем $P P^{-1}$.

Остается:

$$
a_1 = X w = a
$$

**Ответ:**

Значит, предсказания $a_1$ для измененной матрицы признаков равны предсказаниям $a$ для исходной матрицы признаков.

Качество линейной регрессии не изменится.

Связь параметров в исходной задаче и преобразованной следующая:

$$
w_1 = P^{-1} w
$$

Важно еще раз подчеркнуть, что рассуждения справедливы в том случае, если матрица $(X^TX)$ - обратима.

## 3. Алгоритм преобразования

### 3.1 Описание алгоритма

1. Сначала проверим, выполняется ли справедливость наших рассуждений, то есть существует ли обратная матрица для $(X^TX)$. Также добавим столбец единиц в матрицу признаков.

In [10]:
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [11]:
np.linalg.inv(X.T.dot(X))

array([[ 6.87966861e-03, -3.86069397e-04, -9.08230481e-05,
        -8.46490836e-08, -2.37726657e-04],
       [-3.86069397e-04,  8.25500165e-04, -1.04916414e-07,
        -6.32349445e-10,  2.55908411e-06],
       [-9.08230481e-05, -1.04916414e-07,  2.86536437e-06,
         4.32244219e-11,  2.12726300e-07],
       [-8.46490836e-08, -6.32349445e-10,  4.32244219e-11,
         2.07798513e-12,  5.98613139e-10],
       [-2.37726657e-04,  2.55908411e-06,  2.12726300e-07,
         5.98613139e-10,  1.71152624e-04]])

Она существует, значит, можно переходить к следующему шагу.

2. Создаем случайную квадратную матрицу в квадратной размерности количества признаков + 1. Проверяем ее на обратимость.

In [12]:
P = np.random.randint(30, size = (5, 5))
display(P)
display(np.linalg.inv(P))

array([[ 6, 26, 25, 12, 10],
       [ 9, 27, 26, 26, 29],
       [26, 29, 14,  7, 19],
       [26, 19, 26,  8, 25],
       [10, 14, 11, 12, 10]])

array([[-0.0205555 , -0.04702697, -0.00661787,  0.02356362,  0.11059862],
       [ 0.0270545 ,  0.01150202,  0.05457962, -0.0437989 , -0.05461439],
       [ 0.042366  , -0.02329515, -0.04867335,  0.04427438,  0.00698337],
       [-0.02349893, -0.00891439, -0.03667645, -0.01554843,  0.15790699],
       [-0.03572468,  0.06724608,  0.02775882,  0.00771114, -0.13130857]])

3. На этом этапе исходная матрица признаков умножается на обратимую матрицу P: $X_p = XP$ и вычисляются веса и предсказания для новой матрицы.

### 3.2 Обоснование алгоритма

Ниже попробуем обосновать, почему качество модели не должно пострадать.

Сначала рассчитаем вектор весов для исходной матрицы по формуле $w = (X^T X)^{-1} X^T y$.

In [13]:
w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

Затем получим вектор предсказаний по формуле $a = Xw$.

In [14]:
a = X.dot(w)

Затем умножим исходную матрицу на обратимую и получим вектор весов $w_1$ и вектор предсказаний $a_1$.

In [15]:
X_P = X.dot(P)
w1 = np.linalg.inv(X_P.T.dot(X_P)).dot(X_P.T).dot(y)
a1 = X_P.dot(w1)

Следствием сохранения качества модели может служить отсутсвие различий между векторами предсказания $a$ и $a_1$.

Проверим

In [16]:
differences = a1 - a
differences.mean()

1.6515890346336048e-06

Среднее отклонение в векторах предсказаний получилось малым или, возможно, вообще возникло из-за округлений.

Следовательно, мы вправе рассчитывать, что качество моделей линейной регрессии не пострадает после операции защиты данных.

## 4. Проверка алгоритма

Итак, от нас требуется запрограммировать алгоритм преобразования данных. 

Задача поставлена так, что качество модели мы должны проверять на линейной регрессии из sklearn, поэтому не будем создавать собственную модель регрессии, а в алгоритме запрограммируем только проверку обратимости матриц и умножение матрицы признаков на случайную матрицу для защиты.

In [17]:
class DataProtection:
    # Здесь инициализируем матрицу признаков и случайную матрицу, проверим, имеют ли они обратные
    def check(self, features):
        
        self.X_matrix = np.concatenate((np.ones((features.shape[0], 1)), features.values), axis=1)
        self.P_matrix = np.random.randint(100,size = (self.X_matrix.shape[1], self.X_matrix.shape[1]))
        
        if np.linalg.inv(self.X_matrix.T.dot(self.X_matrix)) is not None \
        and np.linalg.inv(self.P_matrix) is not None:
            return 'Матрица признаков и случайная матрица созданы, и имеют обратные матрицы'
        
     # Здесь произведем умножение матрицы признаков на случайную матрицу  
    def transform(self):
        
        X_P_matrix = self.X_matrix.dot(self.P_matrix)
        return X_P_matrix


Начнем проверять алгоритм. Сначала обучим модель на исходных данных.

Разделим данные на обучающую и тестовую выборку.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=2)

Создадим модель линейной регрессии для исходных признаков и проверим ее качество.

In [19]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(f'R_2 = {r2_score(y_test, predictions):.3f}')

R_2 = 0.423


Итак, коэффициент детерминации на исходных данных - 0.423.

Далее приступим к проверке. Инициализируем класс DataProtection в переменной protect и проверим, нет ли ошибок в обратных матрицах.

In [20]:
protect = DataProtection()
protect.check(features)

'Матрица признаков и случайная матрица созданы, и имеют обратные матрицы'

Далее непосредственно осуществим защиту данных, умножив матрицу признаков на случайную обратимую матрицу.

In [21]:
X_P_matrix = protect.transform()

Затем еще раз обучим модель линейной регрессии, но уже на защищенных данных.

Новые данные разобьем на выборки, зафиксировав одинаковый random_state.

In [22]:
X_P_train, X_P_test, y_P_train, y_P_test = train_test_split(X_P_matrix, y, test_size = 0.2, 
                                                            random_state=2 )

Векторы целей для обучающих выборок совпадают:

In [23]:
display(y_train.shape)
sum(y_P_train == y_train)

(3877,)

3877

Создадим модель линейной регрессии для защищенных признаков и проверим качество.

In [24]:
model_P = LinearRegression()
model_P.fit(X_P_train, y_P_train)
predictions_P = model_P.predict(X_P_test)
print(f'R_2 на защищенных данных = {r2_score(y_P_test, predictions_P):.3f}')

R_2 на защищенных данных = 0.423


Коэффициент детерминации для модели линейной регрессии, обученной на защищенных данных, равен тем же 0.423.

Соответственно, защита данных произошла, качество модели не пострадало.